In [1]:
import polars as pl
import numpy as np

In [3]:
airports = pl.read_csv("./../data/airports.csv")
airlines = pl.read_csv("./../data/airlines.csv")
flights = pl.read_csv("./../data/flights.csv")

print("Airports:", airports.shape)
print("Airlines:", airlines.shape)
print("Flights:", flights.shape)


Airports: (322, 7)
Airlines: (14, 2)
Flights: (5819079, 31)


In [4]:
#--renaming and standardizing columns

airports = airports.rename({
    "IATA_CODE": "AIRPORT_CODE",
    "AIRPORT": "AIRPORT_NAME",
    "CITY": "CITY",
    "STATE": "STATE",
    "COUNTRY": "COUNTRY",
    "LATITUDE": "LAT",
    "LONGITUDE": "LON"
})

airlines = airlines.rename({
    "IATA_CODE": "AIRLINE_CODE",
    "AIRLINE": "AIRLINE_NAME"
})

flights = flights.rename({
    "YEAR": "YEAR",
    "MONTH": "MONTH",
    "DAY": "DAY",
    "DAY_OF_WEEK": "DAY_OF_WEEK",
    "AIRLINE": "AIRLINE_CODE",
    "FLIGHT_NUMBER": "FLIGHT_NUM",
    "TAIL_NUMBER": "TAIL_NUM",
    "ORIGIN_AIRPORT": "ORIGIN_CODE",
    "DESTINATION_AIRPORT": "DEST_CODE",
    "SCHEDULED_DEPARTURE": "SCHED_DEP",
    "DEPARTURE_TIME": "DEP_TIME",
    "DEPARTURE_DELAY": "DEP_DELAY",
    "TAXI_OUT": "TAXI_OUT",
    "WHEELS_OFF": "WHEELS_OFF",
    "SCHEDULED_TIME": "SCHED_TIME",
    "ELAPSED_TIME": "ELAPSED_TIME",
    "AIR_TIME": "AIR_TIME",
    "DISTANCE": "DISTANCE",
    "WHEELS_ON": "WHEELS_ON",
    "TAXI_IN": "TAXI_IN",
    "SCHEDULED_ARRIVAL": "SCHED_ARR",
    "ARRIVAL_TIME": "ARR_TIME",
    "ARRIVAL_DELAY": "ARR_DELAY",
    "DIVERTED": "DIVERTED",
    "CANCELLED": "CANCELLED",
    "CANCELLATION_REASON": "CANCEL_REASON",
    "AIR_SYSTEM_DELAY": "AIR_SYS_DELAY",
    "SECURITY_DELAY": "SEC_DELAY",
    "AIRLINE_DELAY": "AIRLINE_DELAY",
    "LATE_AIRCRAFT_DELAY": "LATE_AC_DELAY",
    "WEATHER_DELAY": "WEATHER_DELAY"
})


In [5]:
# -- converting delay columns to float and replacing null values
delay_cols = ["DEP_DELAY", "ARR_DELAY", "AIR_SYS_DELAY", "SEC_DELAY",
              "AIRLINE_DELAY", "LATE_AC_DELAY", "WEATHER_DELAY"]

flights = flights.with_columns([
    flights[col].fill_null(0).cast(pl.Float64) for col in delay_cols
])


# --- Convert Diverted & Cancelled to Boolean
flights = flights.with_columns([
    flights["DIVERTED"].cast(pl.Boolean),
    flights["CANCELLED"].cast(pl.Boolean)
])


# --- Clean cancellation reason ("null" → None)
flights = flights.with_columns(
    pl.when((flights["CANCEL_REASON"] == "null") | (flights["CANCEL_REASON"] == ""))
      .then(None)
      .otherwise(flights["CANCEL_REASON"])
      .alias("CANCEL_REASON")
)

# --- Parse scheduled/actual times (HHMM → datetime)
def parse_time(df, col):
    return (
        df[col]
        .cast(pl.Int32)
        .fill_null(-1)
        .cast(str)
        .str.zfill(4)
        .str.strptime(pl.Datetime, "%H%M", strict=False)
    )

flights = flights.with_columns([
    parse_time(flights, "SCHED_DEP").alias("SCHED_DEP_TIME"),
    parse_time(flights, "DEP_TIME").alias("DEP_TIME_CLEAN"),
    parse_time(flights, "SCHED_ARR").alias("SCHED_ARR_TIME"),
    parse_time(flights, "ARR_TIME").alias("ARR_TIME_CLEAN")
])

print("Cleaned flights:", flights.shape)

Cleaned flights: (5819079, 35)


In [6]:
flights.head()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUM,TAIL_NUM,ORIGIN_CODE,DEST_CODE,SCHED_DEP,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,SCHED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHED_ARR,ARR_TIME,ARR_DELAY,DIVERTED,CANCELLED,CANCEL_REASON,AIR_SYS_DELAY,SEC_DELAY,AIRLINE_DELAY,LATE_AC_DELAY,WEATHER_DELAY,SCHED_DEP_TIME,DEP_TIME_CLEAN,SCHED_ARR_TIME,ARR_TIME_CLEAN
i64,i64,i64,i64,str,i64,str,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,bool,bool,str,f64,f64,f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2015,1,1,4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11.0,21,15,205,194,169,1448,404,4,430,408,-22.0,false,false,null,0.0,0.0,0.0,0.0,0.0,0001-01-01 00:05:00,0001-01-01 23:54:00,0001-01-01 04:30:00,0001-01-01 04:08:00
2015,1,1,4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8.0,12,14,280,279,263,2330,737,4,750,741,-9.0,false,false,null,0.0,0.0,0.0,0.0,0.0,0001-01-01 00:10:00,0001-01-01 00:02:00,0001-01-01 07:50:00,0001-01-01 07:41:00
2015,1,1,4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2.0,16,34,286,293,266,2296,800,11,806,811,5.0,false,false,null,0.0,0.0,0.0,0.0,0.0,0001-01-01 00:20:00,0001-01-01 00:18:00,0001-01-01 08:06:00,0001-01-01 08:11:00
2015,1,1,4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5.0,15,30,285,281,258,2342,748,8,805,756,-9.0,false,false,null,0.0,0.0,0.0,0.0,0.0,0001-01-01 00:20:00,0001-01-01 00:15:00,0001-01-01 08:05:00,0001-01-01 07:56:00
2015,1,1,4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1.0,11,35,235,215,199,1448,254,5,320,259,-21.0,false,false,null,0.0,0.0,0.0,0.0,0.0,0001-01-01 00:25:00,0001-01-01 00:24:00,0001-01-01 03:20:00,0001-01-01 02:59:00


In [7]:
# --- merge datasets ---

flights = flights.join(airlines, on="AIRLINE_CODE", how="left")

flights = flights.join(
    airports.select(["AIRPORT_CODE", "AIRPORT_NAME", "CITY", "STATE"]),
    left_on="ORIGIN_CODE", right_on="AIRPORT_CODE", how="left"
).rename({
    "AIRPORT_NAME": "ORIGIN_AIRPORT_NAME",
    "CITY": "ORIGIN_CITY",
    "STATE": "ORIGIN_STATE"
})


flights = flights.join(
    airports.select(["AIRPORT_CODE", "AIRPORT_NAME", "CITY", "STATE"]),
    left_on="DEST_CODE", right_on="AIRPORT_CODE", how="left"
).rename({
    "AIRPORT_NAME": "DEST_AIRPORT_NAME",
    "CITY": "DEST_CITY",
    "STATE": "DEST_STATE"
})


print("Final flights dataset:", flights.shape)

Final flights dataset: (5819079, 42)


In [8]:
flight_data = flights

In [15]:
# -- data summary -- 

print(
   flight_data.select([
    pl.col("AIRLINE_NAME").n_unique().alias("Unique Airlines"),
    pl.col("ORIGIN_CODE").n_unique().alias("Unique Origin Airports"),
    pl.col("DEST_CODE").n_unique().alias("Unique Destination Airports"),
    pl.col("FLIGHT_NUM").count().alias("Total Flights")
])
)

shape: (1, 4)
┌─────────────────┬────────────────────────┬─────────────────────────────┬───────────────┐
│ Unique Airlines ┆ Unique Origin Airports ┆ Unique Destination Airports ┆ Total Flights │
│ ---             ┆ ---                    ┆ ---                         ┆ ---           │
│ u32             ┆ u32                    ┆ u32                         ┆ u32           │
╞═════════════════╪════════════════════════╪═════════════════════════════╪═══════════════╡
│ 14              ┆ 628                    ┆ 629                         ┆ 5819079       │
└─────────────────┴────────────────────────┴─────────────────────────────┴───────────────┘


In [17]:
flight_data.write_csv("./../data/cleaned_flight_data.csv")